Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

do this first
in the anaconda prompt  or in the Google Colab

pip install pandas-datareader
pip install yfinance

In [47]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np 
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [48]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".
start = dt.datetime(2020,9,22)
end = dt.datetime.now()

stock_symbol = ['AVAX-USD']

In [49]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.
deletd_column =[]
df = pdr.get_data_yahoo(stock_symbol, start, end).dropna()
df1 = df[["Open","Adj Close"]]

#df = #TODO
#df.head()
#df.shape
df1.columns
#df["Open"]

[*********************100%***********************]  1 of 1 completed


Index(['Open', 'Adj Close'], dtype='object')

In [50]:
df1

,Open,Adj Close
Date,,
2020-09-22,4.986754,5.234632
2020-09-23,5.234104,3.521812
2020-09-24,3.521812,4.578369
2020-09-25,4.585006,4.711471
2020-09-26,4.725395,4.541182
...,...,...
2021-11-03,67.773087,75.430420
2021-11-04,75.577637,79.293221
2021-11-05,79.047600,74.364738


In [56]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df1['Close_15_Rolling'] = df1['Adj Close'].rolling(16).mean() #start from zero ?

df1['Open_1_Change'] = df1['Open'].diff()

df1['Adj Close_Change'] = df1['Adj Close'].diff()

df1=df1.dropna()
X=df1
X

<ipython-input-56-5346e52bfa49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Close_15_Rolling'] = df1['Adj Close'].rolling(16).mean() #start from zero ?
<ipython-input-56-5346e52bfa49>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Open_1_Change'] = df1['Open'].diff()
<ipython-input-56-5346e52bfa49>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,Open,Adj Close,Close_15_Rolling,Open_1_Change,Adj Close_Change
Date,,,,,
2020-10-28,4.122243,4.066606,4.113348,-0.038730,-0.055637
2020-10-29,4.066606,3.775312,4.093313,-0.055637,-0.291294
2020-10-30,3.775312,3.714938,4.066118,-0.291294,-0.060374
2020-10-31,3.714938,3.648368,4.036181,-0.060374,-0.066570
2020-11-01,3.648368,3.682664,4.013818,-0.066570,0.034296
...,...,...,...,...,...
2021-11-03,67.773087,75.430420,64.832815,2.804817,7.845863
2021-11-04,75.577637,79.293221,66.237581,7.804550,3.862801
2021-11-05,79.047600,74.364738,67.162887,3.469963,-4.928482


In [55]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
#np.where(a < 5, a, 10*a)
y = X["Adj Close_Change"]
#y = np.where(y<0,y=1)
y

Date
2020-10-11     4.095879
2020-10-14     4.150056
2020-10-15     4.127353
2020-10-16     4.040480
2020-10-17     4.196009
                ...    
2021-11-03    75.430420
2021-11-04    79.293221
2021-11-05    74.364738
2021-11-06    78.685204
2021-11-08    95.372963
Name: Adj Close, Length: 391, dtype: float64

In [53]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

#index = int(0.7*len(X))
#X_train, X_test, y_train, y_test = #TODO

In [54]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = #TODO

SyntaxError: invalid syntax (<ipython-input-54-48dbe8ec2450>, line 6)

In [ ]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

In [ ]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = #TODO
predictions = #TODO

In [ ]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

#TODO

In [ ]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

#TODO

In [ ]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = #TODO